## Collecting Data 

To collect my Spotify Data I used this source: https://github.com/plamere/spotipy/tree/master/examples

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time 


client_id = '3d5be77a0a8b4b34a19fe1c509719169'
client_secret = 'b5f6d556e9a74f48a88ec4e4a40bfc5e'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [2]:

def getTrackIDs(user, playlist_id):
    ids = []
    playlist = sp.user_playlist(user, playlist_id)

    for item in playlist['tracks']['items']:
        track = item['track']
        ids.append(track['id'])
        
    
    return ids



ids = getTrackIDs('ma music', '44ml9I9xyPyRzAy3gWhSAX')


def getTrackFeatures(id):
  meta = sp.track(id)
  features = sp.audio_features(id)

  # meta
  name = meta['name']
  album = meta['album']['name']
  artist = meta['album']['artists'][0]['name']
  length = meta['duration_ms']
  popularity = meta['popularity']
 
  # features
  acousticness = features[0]['acousticness']
  danceability = features[0]['danceability']
  energy = features[0]['energy']
  instrumentalness = features[0]['instrumentalness']
  mode = features[0]['mode']
  liveness = features[0]['liveness']
  loudness = features[0]['loudness']
  speechiness = features[0]['speechiness']
  tempo = features[0]['tempo']
  time_signature = features[0]['time_signature']
  valence = features[0]['valence']

  track = [name, album, artist, length, popularity, danceability, acousticness, energy, instrumentalness, mode, liveness, loudness, speechiness, tempo, time_signature, valence]
  return track



# loop over track ids 
tracks = []
for i in range(len(ids)):
  time.sleep(.5)
  track = getTrackFeatures(ids[i])
  tracks.append(track)

# create dataset
tracks_df = pd.DataFrame(tracks, columns = ['trackname', 'album', 'artist', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', ' mode', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence'])

In [3]:
dct = sp.playlist("44ml9I9xyPyRzAy3gWhSAX", fields=None, market=None, additional_types=('track', ))
result = sp.search(dct['tracks']['items'][0]['track']['artists'][0]['name'])
track = result['tracks']['items'][0]

artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
print("artist genres:", artist["genres"])

genres = artist["genres"]
artist = artist['name']
print(genres)
print (artist)

artist genres: ['francoton', 'french hip hop', 'french pop', 'pop urbaine']
['francoton', 'french hip hop', 'french pop', 'pop urbaine']
Dadju


In [4]:
from tqdm import tqdm
import pandas as pd

In [5]:
rows = []
for t in tqdm(range(len(tracks_df))):
    track_row = []
    
    track = dct['tracks']['items'][t]['track']['name']
    artist = dct['tracks']['items'][t]['track']['artists'][0]['name']
    
    result = sp.search(dct['tracks']['items'][t]['track']['artists'][0]['name'])
    tr = result['tracks']['items'][0]
    art = sp.artist(tr["artists"][0]["external_urls"]["spotify"])
    
    genre = art["genres"]
    try:
        track_row = [artist, genre[0]]
    except:
        track_row = [artist]
    rows.append(track_row)
    
    
genre = pd.DataFrame.from_dict(rows)

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:03<00:00,  7.90it/s]


In [6]:
genre.columns = ['artist', 'genres']

In [7]:
genre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  26 non-null     object
 1   genres  26 non-null     object
dtypes: object(2)
memory usage: 544.0+ bytes


In [8]:
df = pd.merge(tracks_df, genre,  on = 'artist') #"artist" is the common identifier
df.head()

,trackname,album,artist,length,popularity,danceability,acousticness,energy,instrumentalness,mode,liveness,loudness,speechiness,tempo,time_signature,valence,genres
0,Sous contrôle,Gentleman 2.0,Dadju,182560,50,0.676,0.328,0.729,0.000000,0,0.1060,-6.172,0.2350,145.987,4,0.712,francoton
1,Dalida,Fruit du démon,Soolking,225373,47,0.753,0.660,0.728,0.000013,0,0.0889,-4.676,0.0449,140.129,4,0.405,francoton
2,Dalida,Fruit du démon,Soolking,225373,47,0.753,0.660,0.728,0.000013,0,0.0889,-4.676,0.0449,140.129,4,0.405,francoton
3,Dalida,Fruit du démon,Soolking,225373,47,0.753,0.660,0.728,0.000013,0,0.0889,-4.676,0.0449,140.129,4,0.405,francoton
4,Dalida,Fruit du démon,Soolking,225373,47,0.753,0.660,0.728,0.000013,0,0.0889,-4.676,0.0449,140.129,4,0.405,francoton


In [9]:
grouped = df.groupby(['trackname','artist'], as_index=True).size()
grouped[grouped > 1].count()

21

In [10]:
df.drop_duplicates(subset=['trackname','artist'], inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 0 to 229
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   trackname         26 non-null     object 
 1   album             26 non-null     object 
 2   artist            26 non-null     object 
 3   length            26 non-null     int64  
 4   popularity        26 non-null     int64  
 5   danceability      26 non-null     float64
 6   acousticness      26 non-null     float64
 7   energy            26 non-null     float64
 8   instrumentalness  26 non-null     float64
 9    mode             26 non-null     int64  
 10  liveness          26 non-null     float64
 11  loudness          26 non-null     float64
 12  speechiness       26 non-null     float64
 13  tempo             26 non-null     float64
 14  time_signature    26 non-null     int64  
 15  valence           26 non-null     float64
 16  genres            26 non-null     object 
dty

In [12]:
df.to_csv("playlist2.csv", sep = ',')